In [1]:
import pythran
%load_ext pythran.magic

In [2]:
def get_complex_random_array(dimensions):
    import numpy as np
    real_part = np.random.rand(*dimensions)
    imaginary_part = np.random.rand(*dimensions)
    complex_array = real_part + imaginary_part * 1.0j
    return complex_array

In [3]:
import numpy as np
from numba import complex64, prange, jit, types

@jit(['types.UniTuple(complex64[:, :, :], 3)(complex64[:, :, :], complex64[:, :, :], complex64[:, :, :], complex64[:, :, :])'],
     nopython=True, cache=True, nogil=True, fastmath=True, parallel=True)
def numba_multiple_on_exp(integrand_x, integrand_y, integrand_z, kr):
    shape = kr.shape
    result_x = np.zeros((shape[0], shape[1], shape[2]), dtype=type(kr[0, 0, 0]))
    result_y = np.zeros((shape[0], shape[1], shape[2]), dtype=type(kr[0, 0, 0]))
    result_z = np.zeros((shape[0], shape[1], shape[2]), dtype=type(kr[0, 0, 0]))
    
    for j in prange(shape[1]):
        for k in range(shape[2]):
            integr_x = integrand_x[0, j, k]
            integr_y = integrand_y[0, j, k]
            integr_z = integrand_z[0, j, k]
            for i in range(shape[0]):
                exp_j = (np.cos(np.real(kr[i, j, k])) + 1.0j*np.sin(np.real(kr[i, j, k]))) * \
                                                            np.exp(-np.imag(kr[i, j, k]))
                result_x[i, j, k] = integr_x * exp_j
                result_y[i, j, k] = integr_y * exp_j
                result_z[i, j, k] = integr_z * exp_j
                            
    return result_x, result_y, result_z

In [4]:
import numpy as np
from numba import complex64, jit, types

@jit(['types.UniTuple(complex64[:, :, :], 3)(complex64[:, :, :], complex64[:, :, :], complex64[:, :, :], complex64[:, :, :])'],
     nopython=True, cache=True, nogil=True, fastmath=True)
def numba_multiple_on_exp_1core_mode(integrand_x, integrand_y, integrand_z, kr): # Don't repeat yourself, kids!
    shape = kr.shape
    result_x = np.zeros((shape[0], shape[1], shape[2]), dtype=type(kr[0, 0, 0]))
    result_y = np.zeros((shape[0], shape[1], shape[2]), dtype=type(kr[0, 0, 0]))
    result_z = np.zeros((shape[0], shape[1], shape[2]), dtype=type(kr[0, 0, 0]))
    
    for j in range(shape[1]):
        for k in range(shape[2]):
            integr_x = integrand_x[0, j, k]
            integr_y = integrand_y[0, j, k]
            integr_z = integrand_z[0, j, k]
            for i in range(shape[0]):
                exp_j = (np.cos(np.real(kr[i, j, k])) + 1.0j*np.sin(np.real(kr[i, j, k]))) * \
                                                            np.exp(-np.imag(kr[i, j, k]))
                result_x[i, j, k] = integr_x * exp_j
                result_y[i, j, k] = integr_y * exp_j
                result_z[i, j, k] = integr_z * exp_j
                            
    return result_x, result_y, result_z

In [5]:
%%pythran -fopenmp -Ofast
#pythran export pythran_multiple_on_exp(complex64[:, :, :], complex64[:, :, :], complex64[:, :, :], complex64[:, :, :])

def pythran_multiple_on_exp(integrand_x, integrand_y, integrand_z, kr):
    import numpy as np
    shape = kr.shape
    result_x = np.zeros((shape[0], shape[1], shape[2]), dtype=type(kr[0, 0, 0]))
    result_y = np.zeros((shape[0], shape[1], shape[2]), dtype=type(kr[0, 0, 0]))
    result_z = np.zeros((shape[0], shape[1], shape[2]), dtype=type(kr[0, 0, 0]))
    "omp parallel for"
    for j in range(shape[1]):
        for k in range(shape[2]):
            integr_x = integrand_x[0, j, k]
            integr_y = integrand_y[0, j, k]
            integr_z = integrand_z[0, j, k]
            for i in range(shape[0]):
                exp_j = (np.cos(kr[i, j, k].real) + 1.0j*np.sin(kr[i, j, k].real)) * \
                                                            np.exp(-kr[i, j, k].imag)
                result_x[i, j, k] = integr_x * exp_j
                result_y[i, j, k] = integr_y * exp_j
                result_z[i, j, k] = integr_z * exp_j
                            
    return result_x, result_y, result_z

In [6]:
%%pythran -Ofast
#pythran export pythran_multiple_on_exp_1core_mode(complex64[:, :, :], complex64[:, :, :], complex64[:, :, :], complex64[:, :, :])

def pythran_multiple_on_exp_1core_mode(integrand_x, integrand_y, integrand_z, kr):
    import numpy as np
    shape = kr.shape
    result_x = np.zeros((shape[0], shape[1], shape[2]), dtype=type(kr[0, 0, 0]))
    result_y = np.zeros((shape[0], shape[1], shape[2]), dtype=type(kr[0, 0, 0]))
    result_z = np.zeros((shape[0], shape[1], shape[2]), dtype=type(kr[0, 0, 0]))
    for j in range(shape[1]):
        for k in range(shape[2]):
            integr_x = integrand_x[0, j, k]
            integr_y = integrand_y[0, j, k]
            integr_z = integrand_z[0, j, k]
            for i in range(shape[0]):
                exp_j = (np.cos(kr[i, j, k].real) + 1.0j*np.sin(kr[i, j, k].real)) * \
                                                            np.exp(-kr[i, j, k].imag)
                result_x[i, j, k] = integr_x * exp_j
                result_y[i, j, k] = integr_y * exp_j
                result_z[i, j, k] = integr_z * exp_j
                            
    return result_x, result_y, result_z

In [7]:
def numpy_multiple_on_exp(integrand_x, integrand_y, integrand_z, kr):
    numpy_exponents = np.exp(1j * kr)
    result_x = integrand_x * numpy_exponents
    result_y = integrand_y * numpy_exponents
    result_z = integrand_z * numpy_exponents
    
    return result_x, result_y, result_z

In [9]:
import datetime
import numpy as np

def test_exponents_welding():
    numpy_time = []
    pythran_time = []
    pythran_1core_time = []
    numba_time = []
    numba_1core_time = []
    for i in range(10):
        test_kr = get_complex_random_array((50, 300, 710)).astype(np.complex64)
        integrand_x = get_complex_random_array((1, 300, 710)).astype(np.complex64)
        integrand_y = get_complex_random_array((1, 300, 710)).astype(np.complex64)
        integrand_z = get_complex_random_array((1, 300, 710)).astype(np.complex64)

        #numpy test zone
        numpy_start = datetime.datetime.now()
        processed_x, processed_y, processed_z = \
                    numpy_multiple_on_exp(integrand_x, integrand_y, integrand_z, test_kr)
        numpy_end = datetime.datetime.now()
        numpy_time.append(numpy_end - numpy_start)

        #pythran test zone
        pythran_start = datetime.datetime.now()
        processed_x, processed_y, processed_z = \
                    pythran_multiple_on_exp(integrand_x, integrand_y, integrand_z, test_kr)
        pythran_end = datetime.datetime.now()
        pythran_time.append(pythran_end - pythran_start)
        
        #numba test zone
        numba_start = datetime.datetime.now()
        processed_x, processed_y, processed_z = \
                    numba_multiple_on_exp(integrand_x, integrand_y, integrand_z, test_kr)
        numba_end = datetime.datetime.now()
        numba_time.append(numba_end - numba_start)
        
        #pythran 1 core test zone
        pythran_1core_start = datetime.datetime.now()
        processed_x, processed_y, processed_z = \
                    pythran_multiple_on_exp_1core_mode(integrand_x, integrand_y, integrand_z, test_kr)
        pythran_1core_end = datetime.datetime.now()
        pythran_1core_time.append(pythran_1core_end - pythran_1core_start)
        
        #numba test zone
        numba_1core_start = datetime.datetime.now()
        processed_x, processed_y, processed_z = \
                    numba_multiple_on_exp_1core_mode(integrand_x, integrand_y, integrand_z, test_kr)
        numba_1core_end = datetime.datetime.now()
        numba_1core_time.append(numba_1core_end - numba_1core_start)

    print('numpy time: ', np.sum(numpy_time))
    print('pythran time [3 cores mode]: ', np.sum(pythran_time))
    print('numba time [3 cores mode]: ', np.sum(numba_time))
    print('pythran time [1 core mode]: ', np.sum(pythran_1core_time))
    print('numba time [1 core mode]: ', np.sum(numba_1core_time), '\n')
    print('pythran speed up [3 cores mode] = ', np.sum(numpy_time) / np.sum(pythran_time))
    print('numba speed up [3 cores mode] = ', np.sum(numpy_time) / np.sum(numba_time))
    print('pythran speed up [1 core mode] = ', np.sum(numpy_time) / np.sum(pythran_1core_time))
    print('numba speed up [1 core mode] = ', np.sum(numpy_time) / np.sum(numba_1core_time))
    
test_exponents_welding()

numpy time:  0:00:02.942461
pythran [3 cores mode] time:  0:00:01.230795
numba [3 cores mode] time:  0:00:02.281194
pythran [1 core mode] time:  0:00:03.041358
numba [1 core mode] time:  0:00:05.390068 

pythran speed up [3 cores mode] =  2.3906995072290673
numba speed up [3 cores mode] =  1.2898775816524153
pythran speed up [1 core mode] =  0.9674826179621078
numba speed up [1 core mode] =  0.5459042446217747


Above you can see results from Ubuntu on VS with 3 threads (Ryzen 5 3500x)

Results on 4-cores machine: <br/>
&emsp;numpy time:  0:00:04.36<br/>
&emsp;pythran time [4 cores mode]:  0:00:01.037<br/>
&emsp;numba time [4 cores mode]:  0:00:01.45<br/>
&emsp;pythran time [1 core mode]:  0:00:03.54<br/>
&emsp;numba time [1 core mode]:  0:00:03.49 <br/>

&emsp;pythran speed up [4 cores mode] =  4.21<br/>
&emsp;numba speed up [4 cores mode] =  3.00<br/>
&emsp;pythran speed up [1 core mode] =  1.23<br/>
&emsp;numba speed up [1 core mode] =  1.25<br/>

<br/>

Results on 32-cores machine<br/>
&emsp;numpy time:  0:00:15.069883<br/>
&emsp;pythran time [32 cores mode]:  0:00:02.10<br/>
&emsp;numba time [32 cores mode]:  0:00:12.84<br/>
&emsp;pythran time [1 core mode]:  0:00:16.74<br/>
&emsp;numba time [1 core mode]:  0:00:17.20 <br/>

&emsp;pythran speed up [32 cores mode] =  7.19<br/>
&emsp;numba speed up [32 cores mode] =  1.17<br/>
&emsp;pythran speed up [1 core mode] =  0.90<br/>
&emsp;numba speed up [1 core mode] =  0.88<br/>

It was applied to optimize the ```electric field``` method inside ```expansion.py``` file of Smuthi python module. Tests on different machines give results: # TODO Add description, because I can forget it.

4 cores: laptop (I7 8550U (1.8 GHz in 4-cores mode / 4 GHz in 1-core mode), Ubuntu 20.04)


6 cores: computer (Ryzen 5 3500x, Windows 10)


32 cores: computer (2 x E5-2697A v4 @ 2.60GHz (Turboboost 3.6 GHz), Ubuntu 16.04)


3 cores (threads on VM): (Ryzen 5 3500x, Ubuntu 20.10)


|     Way                         | 3 cores VM  |   4 cores     |  32 cores   |   6 cores   |
| :---                            |    :----:   |     :----:    |    :----:   |        ---: |
| December version                | 476         | 886           | 951         | 579         |
| Current version (total)         | 139         | 207           | 316         | 256         |
| Speed up (compared to december) | 3,4         | 4,3           | 3,0         | 2,3         |
| Current version ("welded" part) | 104         | 148           | 211         | 205         |
|                                                                                           |
| Numba way (total)               | 111         | 178           | 253         | 84          |
| Numba way (welded part)         | 73          | 71            | 153         | 33          |
| Numba speed up of welded part   | 1,4         | 2,1           | 1,3         | 6,2         |
|                                                                                           |
| Pythran way (total)             | 82          | 115           | 128         | ---         |
| Pythran way (welded part)       | 39          | 39            | 20          | ---         |
| Pythran speed up of welded part | 2.7         | 3,8           | 10,1        | ---         |
|                                                                                           |
| Parallel by mp.Process          | 77          | 127           | 102         | CRUSH       |
| Parallel speed up               | 1.8         | 1.6           | 3.2         | CRUSH       |